<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introducción" data-toc-modified-id="Introducción-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introducción</a></span></li><li><span><a href="#Configuración" data-toc-modified-id="Configuración-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Configuración</a></span></li><li><span><a href="#Definición-de-una-herramienta-en-RAPID" data-toc-modified-id="Definición-de-una-herramienta-en-RAPID-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Definición de una herramienta en RAPID</a></span></li><li><span><a href="#Definición-de-WorkObjects" data-toc-modified-id="Definición-de-WorkObjects-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Definición de WorkObjects</a></span></li><li><span><a href="#Desarrollo-de-una-aplicación(Parte-I)" data-toc-modified-id="Desarrollo-de-una-aplicación(Parte-I)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Desarrollo de una aplicación(Parte I)</a></span></li><li><span><a href="#Desarrollo-de-una-aplicación(Parte-II)" data-toc-modified-id="Desarrollo-de-una-aplicación(Parte-II)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Desarrollo de una aplicación(Parte II)</a></span></li></ul></div>

# Introducción

# Configuración

In [1]:
import numpy as np
import pandas as pd

# MATLAB API
import matlab.engine

In [2]:
import io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = r'{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())

In [3]:
eng = matlab.engine.start_matlab() # start matlab session
eng.eval("format compact", nargout=0) # compact formatting in matlab's stdout

# Definición de una herramienta en RAPID

Para definir la herramienta, podemos utilzar el tipo de dato `tooldata`. Como el robot sostiene la herramienta `robhold` será `TRUE`. Para la componente `tframe` definiremos la posicion del TCP (en mm), expresada en el sistema de coordenadas de la muñeca `tool0`, así como su orientación. Por último la componente `loaddata`, que caracteriza los parametros dinámicos de la herramienta, viene definida a su vez por cuatro subcomponentes: `mass` para la masa(en kg), `cog` para el centro de gravedad(en mm), `aom` para la orientación de los ejes de inercia e `ix` `iy` `iz` para los valores de dichas inercias(en $kgm^2$).
<br><br>
Así pues, la estructura desglosada de `tooldata` tiene el siguiente aspecto:

```
< dataobject of tooldata >
    < robhold of bool >
        < tframe of pose >
        < trans of pos >
            < x of num >
            < y of num >
            < z of num >
        < rot of orient >
            < q1 of num >
            < q2 of num >
            < q3 of num >
            < q4 of num >
    < tload of loaddata >
        < mass of num >
        < cog of pos >
            < x of num >
            < y of num >
            < z of num >
        < aom of orient >
            < q1 of num >
            < q2 of num >
            < q3 of num >
            < q4 of num >
        < ix of num >
        < iy of num >
        < iz of num >
```

Se nos pide lo siguiente:
<img style="float: right; padding-right: 80px; padding-top: 40px;" src="./images/ejemplo_puntero.png">
>A la vista de la estructura del tipo de dato tooldata, defínase sobre el
papel una herramienta de tipo puntero, considerando una punta de **105
mm de longitud** y **140 gramos de peso**, con su centro de gravedad
situado a **20 mm de la base** y con unas inercias en su cdg equivalentes
a las que tendría una **varilla del mismo peso y longitud respecto de su
cdg**. En cuanto a la orientación del TCP, se pide definirla a través del
correspondiente cuaternio de **2 formas diferentes**:
- Invirtiendo la orientación de los ejes X y Z del TCP por defecto, pero manteniendo la orientación del eje Y.
- Invirtiendo la orientación del eje Z del TCP por defecto e
intercambiando los ejes X e Y.

<br>
donde se han resaltao algunos parámetros que usaremos para la definición de la herramienta. La varilla, supuesta infinitamente delgada a efectos de cálculos, tiene por lo tanto inercia cero en el eje longitudinal $z$ y, en los ejes transversales $x$ e $y$ viene dada por:
<br>
$$I_x=I_y=\frac{m\ell^{2}}{12}=\frac{0.140\times 0.02^{2}}{12}= 4.66 \times10^{-6} kgm^{2}$$
<br>
donde $m$ y $\ell$ son la masa y la longitud del centro de masa a la base, respectivamente.
<br><br>
RAPID pide al usuario expresar la orientación en forma de quaternios, por lo tanto, distinguiendo entre las dos opciones:<br>

> - Invirtiendo la orientación de los ejes X y Z del TCP por defecto, pero manteniendo la orientación del eje Y.

Que podemos traducir en una rotación de $\frac{\pi}{2}$ sobre el eje $y$. El quaternio viene dado por:
$$ \mathbf{Q}_1 =\left [ \textrm{cos}\frac{\theta}{2}, \mathbf{k}\, \textrm{sin}\frac{\theta}{2} \right ] = \left [ \textrm{cos}\frac{\pi}{4}, \left( 0, 1, 0\right)\, \textrm{sin}\frac{\pi}{4} \right ]= \left [ 0.707,\, 0,\, 0.707,\, 0\right ] $$

> - Invirtiendo la orientación del eje Z del TCP por defecto e intercambiando los ejes X e Y.

Que podriamos expresar como composición de cuaternios. Sin embargo, es cómodo expresar una matriz de rotación y hallar las componentes de $\mathbf{Q}_2$:
$$ \mathbf{R} = \left [ \mathbf{n\,o\,a}\right ] =
\begin{bmatrix}
0 &1  &0 \\ 
1 &0  &0 \\ 
0 &0  &-1 
\end{bmatrix} $$
y sabiendo que
<br>
$$ q_{0}=\frac{1}{2} \sqrt{n_{x}+o_{y}+a_{z}+1} $$
$$ q_{1}=\textrm{sign}( o_z-a_y )\frac{1}{2} \sqrt{n_{x}-o_{y}-a_{z}+1} $$
$$ q_{2}=\textrm{sign}( a_x-n_z )\frac{1}{2} \sqrt{-n_{x}+o_{y}-a_{z}+1} $$
$$ q_{3}=\textrm{sign}( n_y-o_x )\frac{1}{2} \sqrt{-n_{x}-o_{y}+a_{z}+1} $$
<br>
donde $\textrm{sign}$ toma el signo de su argumento. Sustituyendo obtenemos
<br>
$$
q_{0}=\frac{1}{2} \sqrt{0+0-1+1}= 0\\
q_{1}=\textrm{sign}( 0-0 )\frac{1}{2} \sqrt{0-0+1+1}=0.707 \\
q_{2}=\textrm{sign}( 0-0 )\frac{1}{2} \sqrt{-0+0+1+1}=0.707 \\
q_{3}=\textrm{sign}( 1-1 )\frac{1}{2} \sqrt{-0-0-1+1}=0 \\
$$
<br>
Y por lo tanto,
$$ \mathbf{Q}_2 = \left [ 0,\, 0.707,\, 0.707,\, 0\right ] $$
<br>
Verifiquemos con MATLAB:
<br>


In [4]:
%%matlab_magic eng

Q1 = eul2quat([0 0 pi/2], 'XYZ')

R = [0 1 0; 1 0 0; 0 0 -1];
Q2 = rotm2quat(R)

Q1 =
    0.7071         0         0    0.7071
Q2 =
         0    0.7071    0.7071         0




Como queriamos demostrar.
<br><br>
Con los datos obtenidos podemos definir ambas herramientas en RAPID:
<br><br>
<p style = "text-indent : 2em;">
    <code> PERS tooldata puntero1 := [ TRUE, [[0, 0, 105], [0.707, 0, 0, 0.707]], [0.14, [0, 0, 20], [1, 0, 0, 0], 4.66e-6, 4.66e-6, 0]];</code>
</p>
<p style = "text-indent : 2em;">
    <code> PERS tooldata puntero2 := [ TRUE, [[0, 0, 105], [0, 0.707, 0.707, 0]], [0.14, [0, 0, 20], [1, 0, 0, 0], 4.66e-6, 4.66e-6, 0]];</code>
</p>
<br>

donde cabe destacar que `aom` usa el quaternio unidad `[1, 0, 0, 0]` porque hemos calculado los momentos de inercia para los ejes originales.
<br><br>

> En cuanto a la herramienta “Garra con puntero”, establézcanse sobre el papel sendos
TCP diferenciados, uno para la funcionalidad de garra (punto medio entre los dedos
en su extremo) y otro asociado al puntero. La localización de su CDG (680 gramos)
es 5, 0, 25 respecto de tool0 (se considerará como una masa puntual).

![garra_puntero](./images/garra_puntero.png)
<br><br>
donde buscamos unos TCP con la orientación que se presenta en la figura:
![tooltcp](./images/tool_tcps.png)
<br><br>
El momento de inercia en una masa puntual viene dado por $I = mr^2$. Así pues, si el CDG esta a $(5,\,0,\,25)$ respecto de `tool0`, se tiene para cada eje:
<br>
$$ I_x=0.68\times0.005^2=1.7\times10^{-5} \,kgm^2
\\ I_y=0.68\times0^2=0 \,kgm^2
\\ I_z=0.68\times0.025^2=4.25\times10^{-4}\,kgm^2 $$
<br><br>
Los cuaternios que expresan orientación de ambos TCP se puede obtener analíticamente de manera idéntica a como se ha explicado anteriormente; Es por ello que usaremos MATLAB para transformar las matrices de rotación:
<br><br>
$$ R_{garra} = \begin{bmatrix} 1&0&0\\0&-1&0\\0&0&-1\end{bmatrix} \qquad
R_{puntero} = \begin{bmatrix} 0&0&-1\\0&1&0\\1&0&0\end{bmatrix} $$
<br><br>
Y, en forma de cuaternios:

In [5]:
%%matlab_magic eng

Rgarra = [1 0 0; 0 -1 0; 0 0 -1];
Q3 = rotm2quat(Rgarra)

Rpuntero = [0 0 -1; 0 1 0; 1 0 0];
Q4 = rotm2quat(Rpuntero)

Q3 =
     0     1     0     0
Q4 =
    0.7071         0   -0.7071         0




Formateando,
<br>
$$ \mathbf{Q}_3 = \left [ 0,\, 1,\, 0,\, 0\right ] $$
$$ \mathbf{Q}_4 = \left [ 0.707,\, 0,\, -0.707,\, 0\right ] $$
<br>
para la garra y el puntero, respectivamente.
<br><br>
Con estos resultados y basándonos en la figura que describe las dimensiones de la herramienta podemos definirlas en RAPID de la siguiente forma:
<br><br>
<p style = "text-indent : 2em;">
    <code> PERS tooldata garra := [ TRUE, [[0, 0, 76], [0, 1, 0, 0]], [0.68, [5, 0, 25], [1, 0, 0, 0], 1.7e-5, 0, 4.25e-4]];</code>
</p>
<p style = "text-indent : 2em;">
    <code> PERS tooldata puntero := [ TRUE, [[80, 0, 23.5], [0.707, 0, -0.707, 0]], [0.68, [5, 0, 25], [1, 0, 0, 0], 1.7e-5, 0, 4.25e-4]];</code>
</p>
<br><br>

# Definición de WorkObjects

> Definición de WorkObjects (a realizar sobre el Flexpendant): RAPID da soporte a una
estructuración básica del entorno del robot en 2 niveles por medio de los WorkObjects. Estos
elementos incorporan la localización de una referencia de usuario respecto del mundo y la
localización de una referencia de objeto respecto de la de usuario:
<br>

RAPID da soporte a una estructura de localización en dos niveles, el primero de una referencia de usuario respecto del mundo y el segundo una referencia de objeto respecto de la de usuario. Podemos definirlos mediante el tipo de dato `wobjdata`, que esta formado por algunos parametros funcionales y por supuesto dos `pose`s, una para cada nivel(arco). Sin embargo, es interesante definirlo en el FlexPendant de forma interactiva.
<br><br>
Como ejemplo, escogiendo una posición y orientación arbtitraria para la caja con tapa del escenario de prueba, procedemos a definir un *WorkObject* solidario a la tapa.

![escenario](./images/escenario.png)
<br><br>
Podemos definir la referencia mediante tres puntos: dos de ellos en el eje $x$ y otro en el eje $y$, tal y como se muestra en las siguientes figuras:

![plano_trazado](./images/plano_trazado1.png)
<br><br>
Una vez creado, podemos ver el resultado en el FlexPendant:

![flexwobj](./images/flexwobj.png)
<br><br>
O también en el codigo fuente de RAPID:
```
...
TASK PERS wobjdata plano_trazado:=[FALSE,TRUE,"",[[1016.76,-794.748,799.798],[0.707107,-1.70166E-07,-1.68899E-10,0.707107]],[[0,0,0],[1,0,0,0]]];
...
```

# Desarrollo de una aplicación(Parte I)

>Créense 2 procedimientos en RAPID (Pinta_cuadrado y Pinta_circunferencia) que
dibujen respectivamente, un cuadrado de 100 mm de lado o su circunferencia circunscrita
(de 100√2 mm de diámetro). Cada procedimiento tendrá al menos como argumento de
llamada, la localización (de tipo robtarget) del centro geométrico de la figura respecto del
WorkObject definido en el punto anterior (plano_trazado). El procedimiento el trazado de la
figura correspondiente deberá atenerse a las siguientes directrices:
...


>Escríbase el programa principal (main), considerando los procedimientos anteriores
y de forma que se atenga a las siguientes directrices:
...

(Ver [enunciado](./Enunciado.pdf) para más información sobre las directrices.)

Se ha elaborado el siguiente código de RAPID, donde se hace abundande uso de la función `RelTool()` para formar los `robtarget` necesarios para dibujar tanto el cuadrado como la circunferencia

```
MODULE Module1
    PERS tooldata Garra_TCP:=[TRUE,[[0,0,150],[0,1,0,0]],[1,[0,0,10],[1,0,0,0],0,0,0]];
    PERS tooldata Puntero_TCP:=[TRUE,[[150,0,30],[0.707106781,0,-0.707106781,0]],[1,[0,0,10],[1,0,0,0],0,0,0]];
    CONST confdata config_fun:=[0,0,0,0];
    CONST confdata config_fuf:=[0,0,0,1];
    CONST extjoint ejes_externos:=[9E+09,9E+09,9E+09,9E+09,9E+09,9E+09];
    CONST jointtarget casa:=[[0,0,0,0,0,0],ejes_externos];
    CONST num distancia:=150;
    TASK PERS wobjdata plano_trazado:=[FALSE,TRUE,"",[[101.172,952.795,450.836],[2.49104E-07,0,0,1]],[[0,0,0],[1,0,0,0]]];

    VAR intnum emergencia;
    
    PROC Pinta_cuadrado(robtarget centro, PERS wobjdata objeto_trabajo, PERS tooldata tool)
        VAR robtarget Paprox;
        VAR robtarget P1;
        VAR robtarget P2;
        VAR robtarget P3;
        VAR robtarget P4; 
        
        Paprox:= RelTool (centro, 0, 0, 50);
        MoveJ Paprox, v100, fine, tool\WObj:=objeto_trabajo;
        
        P1:= RelTool (centro, 50, 50, 0);
        MoveL P1, v100, fine, tool\WObj:=objeto_trabajo;
        
        P2:= RelTool (centro, 50, -50, 0);
        MoveL P2, v100, fine, tool\WObj:=objeto_trabajo;
        
        P3:= RelTool (centro, -50, -50, 0);
        MoveL P3, v100, fine, tool\WObj:=objeto_trabajo;
        
        P4:= RelTool (centro, -50, 50, 0);
        MoveL P4, v100, fine, tool\WObj:=objeto_trabajo;
        
        !Move to complete square
        MoveL P1, v100, fine, tool\WObj:=objeto_trabajo;
        
        !Move to approximation position
        MoveL Paprox, v100, fine, tool\WObj:=objeto_trabajo;
    ENDPROC
    
    PROC Pinta_circunferencia(robtarget centro, PERS wobjdata objeto_trabajo, PERS tooldata tool)
        VAR robtarget Paprox;
        VAR robtarget P1;
        VAR robtarget P2;
        VAR robtarget P3;
        VAR robtarget P4;
        
        Paprox:= RelTool (centro, 0, 0, 50);
        MoveJ Paprox, v100, fine, tool\WObj:=objeto_trabajo;
        
        P1:= RelTool (centro, 0, 100*Sqrt(2), 0);
        P2:= RelTool (centro, 100*Sqrt(2), 0, 0);
        P3:= RelTool (centro, 0, -100*Sqrt(2), 0);
        P4:= RelTool (centro, -100*Sqrt(2), 0, 0);
        
        MoveL P1, v100, fine, tool\WObj:=objeto_trabajo;
        MoveC P2, P3, v100, fine, tool\WObj:=objeto_trabajo;
        MoveC P4, P1, v100, fine, tool\WObj:=objeto_trabajo;
        
        MoveJ Paprox, v100, fine, tool\WObj:=objeto_trabajo;
        
	ENDPROC 
    
    PROC cerrar()
        ! Completa el código fuente
    ENDPROC
 
    PROC abrir()
        ! Completa el código fuente
    ENDPROC

    
    PROC main()
        VAR robtarget ptest := [[150, 150, 0],[1,0,0,0], config_fun, [0,0,0,0,9E9,9E9] ];
        
        MoveAbsJ casa, v100, fine, Puntero_TCP;
        WHILE DI1=0 DO
            IF DI2=0 THEN
                Pinta_cuadrado ptest, plano_trazado, Puntero_TCP;
            ELSE
                Pinta_circunferencia ptest, plano_trazado, Puntero_TCP;
            ENDIF
        ENDWHILE
        MoveAbsJ casa, v100, fine, Puntero_TCP;
    ENDPROC
    
    TRAP emergency_routine
        StopMove \Quick;
        ! Completa el código fuente
        StartMove;
    ENDTRAP
    
ENDMODULE
```

> Repítase el apartado anterior de forma que el trazado de la figura se lleve a cabo en un
plano inclinado respecto el plano de trazado original (por ejemplo 30º). Para ello se sugiere
modificar la definición del robtarget que define la localización del centro, levantándolo
además respecto del objeto, con el fin de no colisionar con él. ¿Qué impacto tiene el uso de
Offs en lugar de RelTool? Explica el porqué de las diferencias.

Para ello, podemos añadir la siguientes líneas que modifica el `robtarget` `ptest`:
```
ptest:= RelTool(ptest, 0, 0, 0\Ry:=30);
ptest:= Offs(ptest, 0, 0, 25);
```
o de forma mas compacta y quizá mas correcta(convención):
```
ptest:= RelTool(ptest, 0, 0, 25\Ry:=30);
```
Que efectivamente inclina el plano que se va a dibujar y aplica un desplazamiento en el eje $z$ **del plano de trazado**.
<br><br>
Cuando se usa `Offs()` en el primer ejemplo, la modificación se lleva a cabo según los ejes de referencia **respecto de los cuales se ha definido el punto que se le pasa como parámetro** y **no** es por lo tanto una traslación relativa del punto a lo largo de sus ejes. En el caso de que el punto y el objeto desde el cual esta referenciado compartan **la misma orientación** entonces el resultado es **equivalente**. En nuestro caso, no se cumple esta condición, por lo que aplicar `RelTool()` tras modificar la orientación levantaría el plano de trazado con respecto a los ejes de ese robtarget, y no del `plano_trazado`. Sin embargo, al usar `Offs()` el plano de trazado se levanta con respecto al sistema de referencia del que está definido `ptest`, que es `plano_trazado`, tal y como se desea. En el segundo ejemplo, con `RelTool()` tanto las traslaciones y rotaciones se llevan a cabo en torno a la orientación del punto que se le pasa como parámetro y, como `ptest` por ahora solo era una traslación desde `plano_trazado` podemos usar una sola línea que haga las dos transformaciones.

>Incorpórese una rutina de interrupción, con el fin de procurar una parada de
emergencia ante la activación de la entrada binaria nº 3 (DI3).

Para ello:
>Declarar una interrupción como dato de tipo intnum

`intnum emergencia;`
<br><br>

>Asociar el disparo de dicha interrupción con la ejecución de la correspondiente rutina
de servicio de interrupción mediante CONNECT

`CONNECT emergencia WITH emergency_routine;`
<br><br>

>Definir las condiciones de disparo de la interrupción, asociándola al nivel alto de la
entrada binaria nº 3, por medio de ISignalDI 

`ISignalDI DI1, 1, emergencia;`
<br><br>
> Programar la rutina de servicio de interrupción (TRAP), de forma que se detenga el
movimiento del robot (úsese StopMove para ello). La ejecución de la rutina de
servicio deberá tener lugar hasta que la señal binaria que activó la interrupción pase a
valer 0 (úsese WaitDI). La reanudación del movimiento del robot tendrá lugar por
medio de la instrucción StartMove (será la última instrucción de la rutina). Con el fin
de hacer la espera más amena, se sugiere incluir la activación de una salida binaria
y/o la escritura de un mensaje en la ventana de explotación del FlexPendant (por
medio de TPWrite)

```
    TRAP emergency_routine
        StopMove \Quick;
        TPWrite "Emergencia, esperando DI3==0";
        WaitDI DI3, 0;
        TPWrite "Reanudando ejecución";
        StartMove;
    ENDTRAP
```
<br><br>

Se muestra a continuación el código completo con las modificaciones ya explicadas:
```
MODULE Module1
    PERS tooldata Garra_TCP:=[TRUE,[[0,0,150],[0,1,0,0]],[1,[0,0,10],[1,0,0,0],0,0,0]];
    PERS tooldata Puntero_TCP:=[TRUE,[[150,0,30],[0.707106781,0,-0.707106781,0]],[1,[0,0,10],[1,0,0,0],0,0,0]];
    CONST confdata config_fun:=[0,0,0,0];
    CONST confdata config_fuf:=[0,0,0,1];
    CONST extjoint ejes_externos:=[9E+09,9E+09,9E+09,9E+09,9E+09,9E+09];
    CONST jointtarget casa:=[[0,0,0,0,0,0],ejes_externos];
    CONST num distancia:=150;
    TASK PERS wobjdata plano_trazado:=[FALSE,TRUE,"",[[101.172,952.795,450.836],[2.49104E-07,0,0,1]],[[0,0,0],[1,0,0,0]]];

    VAR intnum emergencia;
    
    PROC Pinta_cuadrado(robtarget centro, PERS wobjdata objeto_trabajo, PERS tooldata tool)
        VAR robtarget Paprox;
        VAR robtarget P1;
        VAR robtarget P2;
        VAR robtarget P3;
        VAR robtarget P4; 
        
        Paprox:= RelTool (centro, 0, 0, 50);
        MoveJ Paprox, v100, fine, tool\WObj:=objeto_trabajo;
        
        P1:= RelTool (centro, 50, 50, 0);
        MoveL P1, v100, fine, tool\WObj:=objeto_trabajo;
        
        P2:= RelTool (centro, 50, -50, 0);
        MoveL P2, v100, fine, tool\WObj:=objeto_trabajo;
        
        P3:= RelTool (centro, -50, -50, 0);
        MoveL P3, v100, fine, tool\WObj:=objeto_trabajo;
        
        P4:= RelTool (centro, -50, 50, 0);
        MoveL P4, v100, fine, tool\WObj:=objeto_trabajo;
        
        !Move to complete square
        MoveL P1, v100, fine, tool\WObj:=objeto_trabajo;
        
        !Move to approximation position
        MoveL Paprox, v100, fine, tool\WObj:=objeto_trabajo;
    ENDPROC
    
    PROC Pinta_circunferencia(robtarget centro, PERS wobjdata objeto_trabajo, PERS tooldata tool)
        VAR robtarget Paprox;
        VAR robtarget P1;
        VAR robtarget P2;
        VAR robtarget P3;
        VAR robtarget P4;
        
        Paprox:= RelTool (centro, 0, 0, 50);
        MoveJ Paprox, v100, fine, tool\WObj:=objeto_trabajo;
        
        P1:= RelTool (centro, 0, 100*Sqrt(2), 0);
        P2:= RelTool (centro, 100*Sqrt(2), 0, 0);
        P3:= RelTool (centro, 0, -100*Sqrt(2), 0);
        P4:= RelTool (centro, -100*Sqrt(2), 0, 0);
        
        MoveL P1, v100, fine, tool\WObj:=objeto_trabajo;
        MoveC P2, P3, v100, fine, tool\WObj:=objeto_trabajo;
        MoveC P4, P1, v100, fine, tool\WObj:=objeto_trabajo;
        
        MoveJ Paprox, v100, fine, tool\WObj:=objeto_trabajo;
        
	ENDPROC 
    
    PROC cerrar()
        ! Completa el código fuente
    ENDPROC
 
    PROC abrir()
        ! Completa el código fuente
    ENDPROC

    
    PROC main()
        VAR robtarget ptest := [[150, 150, 0],[1,0,0,0], config_fun, [0,0,0,0,9E9,9E9] ];
        ptest:= RelTool(ptest, 0, 0, 71\Ry:=30);
        
        CONNECT emergencia WITH emergency_routine;
        ISignalDI DI3, 1, emergencia;
        
        MoveAbsJ casa, v100, fine, Puntero_TCP;
        WHILE DI1=0 DO
            IF DI2=0 THEN
                Pinta_cuadrado ptest, plano_trazado, Puntero_TCP;
            ELSE
                Pinta_circunferencia ptest, plano_trazado, Puntero_TCP;
            ENDIF
        ENDWHILE
        MoveAbsJ casa, v100, fine, Puntero_TCP;
    ENDPROC
    
    TRAP emergency_routine
        StopMove \Quick;
        TPWrite "Emergencia, esperando DI3==0";
        WaitDI DI3, 0;
        TPWrite "Reanudando ejecución";
        StartMove;
    ENDTRAP
    
ENDMODULE
 ```

# Desarrollo de una aplicación(Parte II)

> Generación del escenario de partida

Creamos la mesa con las nueve piezas tal y como se describe en el enunciado:
![mesa](./images/table.png)

> Habilítese el uso de la garra

Para ello, usamos nuestra salida digital DO1 para activar o desactivar el actuador y las entradas DI4 y DI5 para los sensores de pinza abierta o cerrada. Así pues. Podemos crear dos procedimientos que abran o cierren la pinza:
```
    PROC cerrar()
        SetDo DO1, 1;
        WaitDI DI4, 1;
    ENDPROC
 
    PROC abrir()
        SetDo DO1, 0;
        WaitDI DI5, 1;
    ENDPROC
```

> Inclúyase un procedimiento (coger_y_dejar), de forma que en cada llamada se recoja
un elemento de la matriz del plano de origen, depositándolo en la correspondiente
localización del plano de destino (otra cara de la mesa).

Con un WorkObject `basemesa` en la esquina de la base de nuestra mesa, es sencillo implementar dicho procedimiento teniendo en cuenta los parámetros geométricos y la localización de las piezas que se apoyan en ésta. Así pues, mediante el uso de la garra que acabamos de implementar y la definición de ciertos destinos de paso para evitar colisiones y asegurar un movimiento óptimo, se elabora el siguiente código; donde se han incluido ciertos comentarios que describen el movimiento que recorre el robot:


```
PROC coger_y_dejar(num fila, num columna)
        VAR num lado:= 400;
        VAR robtarget origen;
        VAR robtarget aproxCoger;
        VAR robtarget aproxPieza;
        VAR robtarget enPieza;
        VAR robtarget esquina;
        VAR robtarget aproxDejar;
        VAR robtarget dejando;
        VAR num xcentro;
        VAR num ycentro;
        xcentro:= 50 + 150*(fila-1);
        ycentro:= 50 + 150*(columna-1);
        origen := [[0, 0, 0],[1,0,0,0], [0,0,0,0], [0,0,0,0,9E9,9E9] ];
        
        !Aproximacion a cara_coger
        aproxCoger:= RelTool(origen, lado/2, lado/2,550);
        MoveJ aproxCoger, v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Aproximacion a pieza
        aproxPieza := RelTool(origen, xcentro, ycentro, 400+100);
        MoveL aproxPieza, v500, fine, Garra_TCP\Wobj:=basemesa;
        abrir;
        
        !Bajar coger y subir
        enPieza := RelTool(aproxPieza, 0, 0, -80);
        MoveL enPieza, v500, fine, Garra_TCP\Wobj:=basemesa;
        cerrar;
        MoveL aproxPieza, v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Ir a otra cara
        esquina:=RelTool(aproxPieza, 0, 100+50+150*(3-columna), 0);
        MoveL esquina, v500, fine, Garra_TCP\WObj:=basemesa;
        aproxDejar:=RelTool(esquina, 0, 0, -100-50-150*(columna-1)\Rx:=-90);
        MoveJ aproxDejar, v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Bajar Dejar y subir
        dejando := RelTool(aproxDejar, 0, 0, -80);
        MoveL dejando, v500, fine, Garra_TCP\Wobj:=basemesa;
        abrir;
        WaitTime 0.5;
        MoveL RelTool(aproxDejar,0,0,100), v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Volver a la primera cara
        MoveJ esquina, v500, fine, Garra_TCP\WObj:=basemesa;
        MoveJ aproxCoger, v500, fine, Garra_TCP\WObj:=basemesa;
        
ENDPROC
```

> Complétese el programa principal, para mover toda la distribución organizada de
objetos desde la cara superior de la mesa (en la que se encuentran inicialmente) a una de las
caras laterales de la misma. Obsérvense las siguientes directrices

En el main, recorremos las filas y columnas con dos bucles iterativos:
```
FOR fila FROM 1 TO 3 DO
        FOR columna FROM 1 TO 3 DO
            coger_y_dejar fila, columna;
        ENDFOR
ENDFOR
```

Se presenta el código completo a continuación:

```
MODULE Module1
    PERS tooldata Garra_TCP:=[TRUE,[[0,0,150],[0,1,0,0]],[1,[0,0,10],[1,0,0,0],0,0,0]];
    PERS tooldata Puntero_TCP:=[TRUE,[[150,0,30],[0.707106781,0,-0.707106781,0]],[1,[0,0,10],[1,0,0,0],0,0,0]];
    CONST confdata config_fun:=[0,0,0,0];
    CONST confdata config_fuf:=[0,0,0,1];
    CONST extjoint ejes_externos:=[9E+09,9E+09,9E+09,9E+09,9E+09,9E+09];
    CONST jointtarget casa:=[[0,0,0,0,0,0],ejes_externos];
    CONST num distancia:=150;
    TASK PERS wobjdata plano_trazado:=[FALSE,TRUE,"",[[101.172,952.795,450.836],[2.49104E-07,0,0,1]],[[0,0,0],[1,0,0,0]]];

    VAR intnum emergencia;
    TASK PERS wobjdata basemesa:=[FALSE,TRUE,"",[[997.701,-0.29706,-0.479568],[0.867482,-2.47042E-08,2.31523E-09,0.497469]],[[0,0,0],[1,0,0,0]]];
    
    PROC Pinta_cuadrado(robtarget centro, PERS wobjdata objeto_trabajo, PERS tooldata tool)
        VAR robtarget Paprox;
        VAR robtarget P1;
        VAR robtarget P2;
        VAR robtarget P3;
        VAR robtarget P4; 
        
        Paprox:= RelTool (centro, 0, 0, 50);
        MoveJ Paprox, v500, fine, tool\WObj:=objeto_trabajo;
        
        P1:= RelTool (centro, 50, 50, 0);
        MoveL P1, v500, fine, tool\WObj:=objeto_trabajo;
        
        P2:= RelTool (centro, 50, -50, 0);
        MoveL P2, v500, fine, tool\WObj:=objeto_trabajo;
        
        P3:= RelTool (centro, -50, -50, 0);
        MoveL P3, v500, fine, tool\WObj:=objeto_trabajo;
        
        P4:= RelTool (centro, -50, 50, 0);
        MoveL P4, v500, fine, tool\WObj:=objeto_trabajo;
        
        !Move to complete square
        MoveL P1, v500, fine, tool\WObj:=objeto_trabajo;
        
        !Move to approximation position
        MoveL Paprox, v500, fine, tool\WObj:=objeto_trabajo;
    ENDPROC
    
    PROC Pinta_circunferencia(robtarget centro, PERS wobjdata objeto_trabajo, PERS tooldata tool)
        VAR robtarget Paprox;
        VAR robtarget P1;
        VAR robtarget P2;
        VAR robtarget P3;
        VAR robtarget P4;
        
        Paprox:= RelTool (centro, 0, 0, 50);
        MoveJ Paprox, v500, fine, tool\WObj:=objeto_trabajo;
        
        P1:= RelTool (centro, 0, 100*Sqrt(2), 0);
        P2:= RelTool (centro, 100*Sqrt(2), 0, 0);
        P3:= RelTool (centro, 0, -100*Sqrt(2), 0);
        P4:= RelTool (centro, -100*Sqrt(2), 0, 0);
        
        MoveL P1, v500, fine, tool\WObj:=objeto_trabajo;
        MoveC P2, P3, v500, fine, tool\WObj:=objeto_trabajo;
        MoveC P4, P1, v500, fine, tool\WObj:=objeto_trabajo;
        
        MoveJ Paprox, v500, fine, tool\WObj:=objeto_trabajo;
        
	ENDPROC 
    
    PROC coger_y_dejar(num fila, num columna)
        VAR num lado:= 400;
        VAR robtarget origen;
        VAR robtarget aproxCoger;
        VAR robtarget aproxPieza;
        VAR robtarget enPieza;
        VAR robtarget esquina;
        VAR robtarget aproxDejar;
        VAR robtarget dejando;
        VAR num xcentro;
        VAR num ycentro;
        xcentro:= 50 + 150*(fila-1);
        ycentro:= 50 + 150*(columna-1);
        origen := [[0, 0, 0],[1,0,0,0], [0,0,0,0], [0,0,0,0,9E9,9E9] ];
        
        !Aproximacion a cara_coger
        aproxCoger:= RelTool(origen, lado/2, lado/2,550);
        MoveJ aproxCoger, v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Aproximacion a pieza
        aproxPieza := RelTool(origen, xcentro, ycentro, 400+100);
        MoveL aproxPieza, v500, fine, Garra_TCP\Wobj:=basemesa;
        abrir;
        
        !Bajar coger y subir
        enPieza := RelTool(aproxPieza, 0, 0, -80);
        MoveL enPieza, v500, fine, Garra_TCP\Wobj:=basemesa;
        cerrar;
        MoveL aproxPieza, v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Ir a otra cara
        esquina:=RelTool(aproxPieza, 0, 100+50+150*(3-columna), 0);
        MoveL esquina, v500, fine, Garra_TCP\WObj:=basemesa;
        aproxDejar:=RelTool(esquina, 0, 0, -100-50-150*(columna-1)\Rx:=-90);
        MoveJ aproxDejar, v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Bajar Dejar y subir
        dejando := RelTool(aproxDejar, 0, 0, -80);
        MoveL dejando, v500, fine, Garra_TCP\Wobj:=basemesa;
        abrir;
        WaitTime 0.5;
        MoveL RelTool(aproxDejar,0,0,100), v500, fine, Garra_TCP\WObj:=basemesa;
        
        !Volver a la primera cara
        MoveJ esquina, v500, fine, Garra_TCP\WObj:=basemesa;
        MoveJ aproxCoger, v500, fine, Garra_TCP\WObj:=basemesa;
        
        
	ENDPROC 
    
    PROC cerrar()
        SetDo DO1, 1;
        WaitDI DI4, 1;
    ENDPROC
 
    PROC abrir()
        SetDo DO1, 0;
        WaitDI DI5, 1;
    ENDPROC

    
    PROC main()
        VAR robtarget ptest := [[150, 150, 0],[1,0,0,0], config_fun, [0,0,0,0,9E9,9E9] ];
        ptest:= RelTool(ptest, 0, 0, 71\Ry:=30);
        
        CONNECT emergencia WITH emergency_routine;
        ISignalDI DI3, 1, emergencia;
        
        MoveAbsJ casa, v1000, fine, Puntero_TCP;
        WHILE DI1=0 DO
            IF DI2=0 THEN
                Pinta_cuadrado ptest, plano_trazado, Puntero_TCP;
            ELSE
                Pinta_circunferencia ptest, plano_trazado, Puntero_TCP;
            ENDIF
        ENDWHILE
        
        !coger_y_dejar 2,1;
        FOR fila FROM 1 TO 3 DO
            FOR columna FROM 1 TO 3 DO
                coger_y_dejar fila, columna;
            ENDFOR
        ENDFOR

        !Programa terminado, parar
        StopMove \Quick;
        WaitTime 3600;
        
    ENDPROC
    
    TRAP emergency_routine
        StopMove \Quick;
        TPWrite "Emergencia, esperando DI3==0";
        WaitDI DI3, 0;
        TPWrite "Reanudando ejecución";
        StartMove;
    ENDTRAP
    
ENDMODULE
```